In [1]:
from __future__ import annotations
import pprint
import pandas as pd

import sys
sys.path.append('../')
import doctable

The iris dataset is simply a list of flowers with information about the sepal, petal, and species.

In [2]:
iris_df = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
core = doctable.ConnectCore.open(
    target=':memory:', # use a filename for a sqlite to write to disk
    dialect='sqlite',
    echo=True,
)
core

ConnectCore(target=':memory:', dialect='sqlite', engine=Engine(sqlite:///:memory:), metadata=MetaData())

In [4]:
import datetime

@doctable.table_schema(table_name='iris', slots=True)
class Iris:
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
    species: str
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    updated: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(default=datetime.datetime.utcnow),
    )
    added: datetime.datetime = doctable.Column(
        column_args=doctable.ColumnArgs(
            default=datetime.datetime.utcnow, 
            onupdate=datetime.datetime.utcnow
        )
    )
    
    @classmethod
    def from_row(cls, row: pd.Series):
        return cls(**row)

Iris(sepal_length=1, sepal_width=2, petal_length=3, petal_width=4, species='setosa')

Iris(sepal_length=1, sepal_width=2, petal_length=3, petal_width=4, species='setosa', id=MISSING, updated=MISSING, added=MISSING)

In [5]:
irises = [Iris.from_row(row) for _, row in iris_df.iterrows()]
print(irises[0])

Iris(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='setosa', id=MISSING, updated=MISSING, added=MISSING)


In [6]:
with core.begin_ddl() as emitter:
    itab = emitter.create_table_if_not_exists(Iris)
itab.name
#itab.inspect().get_columns()
import sqlalchemy
sqlalchemy.inspect(itab.table)

type_hint='int', mth=<class 'int'>, mth.__name__='int', mct=<class 'sqlalchemy.sql.sqltypes.Integer'>
type_hint='int', mth=<class 'float'>, mth.__name__='float', mct=<class 'sqlalchemy.sql.sqltypes.Float'>
type_hint='int', mth=<class 'bool'>, mth.__name__='bool', mct=<class 'sqlalchemy.sql.sqltypes.Boolean'>
type_hint='int', mth=<class 'str'>, mth.__name__='str', mct=<class 'sqlalchemy.sql.sqltypes.String'>
type_hint='int', mth=<class 'bytes'>, mth.__name__='bytes', mct=<class 'sqlalchemy.sql.sqltypes.LargeBinary'>
type_hint='int', mth=<class 'datetime.datetime'>, mth.__name__='datetime', mct=<class 'sqlalchemy.sql.sqltypes.DateTime'>
type_hint='int', mth=<class 'datetime.time'>, mth.__name__='time', mct=<class 'sqlalchemy.sql.sqltypes.Time'>
type_hint='int', mth=<class 'datetime.date'>, mth.__name__='date', mct=<class 'sqlalchemy.sql.sqltypes.Date'>
type_hint='int', mth=typing.Any, mth.__name__='Any', mct=<class 'sqlalchemy.sql.sqltypes.PickleType'>
2023-11-05 09:38:53,250 INFO sqlalc

AttributeError: 'str' object has no attribute '__name__'

## Running Queries

### General Queries

In [ ]:
with core.query() as q:
    q.delete(itab, all=True)
    q.insert_single(itab, {
        'sepal_length': 1,'sepal_width': 2,'petal_length': 3,'petal_width': 4,'species': 'setosa'
    })
    print(q.insert_multi.__doc__)
    q.insert_multi(itab, [
        {'sepal_length': 1, 'sepal_width': 2, 'petal_length': 3, 'petal_width': 4, 'species': 'setosa'},
        {'sepal_length': 1, 'sepal_width': 2, 'petal_length': 3, 'petal_width': 4, 'species': 'setosa'},
    ])
    print(q.select(itab).fetchall())

: 

In [ ]:
with core.query() as q:
    # use table.cols to specify which columns to select
    columns = itab.cols('sepal_length', 'sepal_width')
    pprint.pprint(q.select(columns).fetchall())
    
    # use subscript to specify table for each column. use for table joins
    columns = [itab['sepal_length'], itab['sepal_width']]
    results = q.select(columns).fetchall()
    pprint.pprint(results)
    
    # use .sum(), .min(), .max(), .count(), .sum(), and .unique() to specify aggregate functions
    columns = [itab['species'].distinct()]
    result = q.select(columns).scalars()
    pprint.pprint(f'{result=}')
    
    # use in conjunction with group_by to specify groupings
    columns = [itab['sepal_length'].sum()]
    result = q.select(columns, group_by=[itab['species']]).scalar_one()
    pprint.pprint(f'{result=}')

: 

#### Table-specific Queries

In [ ]:
with itab.query() as q:
    pprint.pprint(q.select())

: 

In [ ]:
with itab.query() as q:
    q.delete(all=True)
    
    q.insert_multi(irises)
    
    db_irises = q.select()
    print(len(db_irises))
    pprint.pprint(db_irises[:2])

: 

In [ ]:
with itab.query() as q:
    db_irises = q.select(['id', 'sepal_width', 'sepal_length'])
    pprint.pprint(db_irises[:2])

: 

## Working with Multple Tables

In [ ]:
print(iris_df['species'].unique())

species_data = {
    'setosa':'bristle-pointed iris',
    'versicolor':'Southern blue flag',
    'virginica':'Northern blue flag',
}

: 

In [ ]:
import typing

@doctable.table_schema(table_name='species')
class Species:
    name: str
    common_name: str = doctable.Column(
        column_args=doctable.ColumnArgs(nullable=True),
    )
    
    id: int = doctable.Column(# will appear as the first column in the table
        column_args=doctable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    
    @classmethod
    def from_dict(cls, data: typing.Dict[str, str]) -> typing.List[Species]:
        return [cls(name=n, common_name=cn) for n,cn in data.items()]

@doctable.table_schema(table_name='iris')
class IrisEntry:
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
    species: str = doctable.Column(
        column_args=doctable.ColumnArgs(foreign_key='species.name', nullable=False),
    )
    
    id: int = doctable.Column(# will appear as the first column in the table
        column_args=doctable.ColumnArgs(order=0, primary_key=True, autoincrement=True),
    )
    
    @classmethod
    def from_dataframe(cls, df: pd.DataFrame) -> typing.List[IrisEntry]:
        return [cls(**row) for _,row in df.iterrows()]

with core.begin_ddl() as emitter:
    #spec_tab = emitter.create_table_if_not_exists(Species)
    iris_tab = emitter.create_table_if_not_exists(IrisEntry)
print(iris_tab)

: 